In [1]:
import time
import random
import numpy as np
from itertools import product
import joblib
import pygame
from environments import atari_environments
from atari_class_functions import aqn_set_state

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
ENVIRONMENT_HEIGHT, ENVIRONMENT_WIDTH = 600, 840
PADDLE_HEIGHT, PADDLE_WIDTH = 110, 10
BALL_SIZE = 10

env_params = {
    "HEIGHT":ENVIRONMENT_HEIGHT,
    "WIDTH":ENVIRONMENT_WIDTH,
    "PADDLE_HEIGHT":PADDLE_HEIGHT,
    "PADDLE_WIDTH":PADDLE_WIDTH,
    "BALL_SIZE":10,
    "BALL_SPEED":(random.choice([-7,7]),random.choice([-7,7])),
    "MISS_REWARD":-5,
    "HIT_REWARD":10,
    "PASSIVE_REWARD":1,
    "RENDER_MODE":False
}
env = atari_environments.pong_env(**env_params)

#### AQN

In [3]:
# Load AQN
AQN = joblib.load("artifacts/models/aqn.sav")

# Ball Position Threshold To start Taking Actions
__xth__, __yth__ = 4, 4

# All Possible States
__states__ = list(product(list(range(0,5)),list(range(0,5)),list(range(0,5)),list(range(0,2)),list(range(0,2))))

alpha = 0.3
gamma = 0.1
epsilon = 0.05

In [4]:
running = True
env.reset()    
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Select and Take Action
    state = aqn_set_state(env)
    # psi = __states__.index(state) # previous state index
    q = __states__.index(state)
    q = AQN["Q_mat"][q,:]
    n = random.random()
    if n <= epsilon:
        action_index = random.randint(0,len(env.__ACTION_SPACE__)-1)
    else:
        action_index = np.argmax(q)
    prev_state,action,next_state,reward = env.step(action_index)

    # state = aqn_set_state(env)
    # nsi = __states__.index(state) #next/current state index
    # AQN["Q_mat"][psi,action_index] = AQN["Q_mat"][psi,action_index] + alpha*(reward+(gamma*max(AQN["Q_mat"][nsi,:]))-AQN["Q_mat"][psi,action_index])
    
    # Render the current state
    env.render()
    
    # Cap the frame rate to avoid maxing out the CPU
    env.clock.tick(60)  # 60 FPS
    
    # Sleep briefly to allow Jupyter to handle other tasks
    time.sleep(0.01)

pygame.quit()